Khai báo các thư viện cần thiết

In [33]:
import os
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

Khai báo đường dẫn tệp dữ liệu

In [34]:
data_path = 'D:/Code/python/mat3533/project/data/'

train_images_path = os.path.join(data_path, 'train-images-idx3-ubyte.gz')
train_labels_path = os.path.join(data_path, 'train-labels-idx1-ubyte.gz')

test_images_path = os.path.join(data_path, 't10k-images-idx3-ubyte.gz')
test_labels_path = os.path.join(data_path, 't10k-labels-idx1-ubyte.gz')

Hàm lấy dữ liệu

In [36]:
def get_mnist_data(images_path, labels_path, num_images, shuffle=False, _is=True, image_size=28):
    """
    This shuffle param is active when .gz is downloaded at:
    - 'http://yann.lecun.com/exdb/mnist/'
    - This function return random num_images in 60000 or 10000
    """
    # read data
    import gzip # to decompress gz (zip) file
    # open file training to read training data
    f_images = gzip.open(images_path,'r')
    # skip 16 first bytes because these are not data, only header infor
    f_images.read(16)
    # general: read num_images data samples if this parameter is set;
    # if not, read all (60000 training or 10000 test)
    real_num = num_images if not shuffle else (60000 if _is else 10000)
    # read all data to buf_images (28x28xreal_num)
    buf_images = f_images.read(image_size * image_size * real_num)
    # images
    images = np.frombuffer(buf_images, dtype=np.uint8).astype(np.float32)
    images = images.reshape(real_num, image_size, image_size,)
    # Read labels
    f_labels = gzip.open(labels_path,'r')
    f_labels.read(8)
    labels = np.zeros((real_num)).astype(np.int64)
    # rearrange to correspond the images and labels
    for i in range(0, real_num):
        buf_labels = f_labels.read(1)
        labels[i] = np.frombuffer(buf_labels, dtype=np.uint8).astype(np.int64)

    # shuffle to get random images data
    if shuffle is True:
        rand_id = np.random.randint(real_num, size=num_images)
        images = images[rand_id, :]
        labels = labels[rand_id,]
    # change images data to type of vector 28x28 dimentional
    images = images.reshape(num_images, image_size * image_size)
    return images, labels

Lấy dữ liệu train, test từ tệp dữ liệu

In [37]:
train_images, train_labels = get_mnist_data(
    train_images_path, train_labels_path, 60000)
test_images, test_labels = get_mnist_data(
    test_images_path, test_labels_path, 10000, _is=False)
print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

(60000, 784) (60000,)
(10000, 784) (10000,)


In [38]:
all_images = np.concatenate((train_images, test_images), axis=0)

In [40]:
pca = PCA(n_components=100)
pca.fit(all_images)
X_train = pca.transform(train_images)
X_test = pca.transform(test_images)
print(X_train.shape)
print(X_test.shape)

(60000, 100)
(10000, 100)


In [41]:
from keras.utils import to_categorical

y_train = to_categorical(train_labels, num_classes=10)
y_test = to_categorical(test_labels, num_classes=10)

In [42]:
from keras.models import Sequential
from keras.layers import Dense, Activation

In [43]:
model = Sequential()
model.add(Dense(10, activation='sigmoid', input_shape=(100,)))
model.add(Dense(10, activation='softmax'))

c:\Users\Admin\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [44]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 10)             │         1,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │           110 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,120 (4.38 KB)

 Trainable params: 1,120 (4.38 KB)

 Non-trainable params: 0 (0.00 B)

In [45]:
model.compile(loss="categorical_crossentropy",
              optimizer="sgd",
              metrics = ['accuracy'])

In [46]:
model.fit(X_train, y_train, batch_size=100, epochs=10)

Epoch 1/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 662us/step - accuracy: 0.2633 - loss: 2.1264
Epoch 2/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 0s 724us/step - accuracy: 0.6202 - loss: 1.5659
Epoch 3/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 0s 709us/step - accuracy: 0.7057 - loss: 1.3059
Epoch 4/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 0s 789us/step - accuracy: 0.7492 - loss: 1.1306
Epoch 5/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 0s 692us/step - accuracy: 0.7764 - loss: 1.0019
Epoch 6/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 0s 737us/step - accuracy: 0.7893 - loss: 0.9115
Epoch 7/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 0s 694us/step - accuracy: 0.8066 - loss: 0.8332
Epoch 8/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 0s 669us/step - accuracy: 0.8194 - loss: 0.7762
Epoch 9/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 0s 646us/step - accuracy: 0.8245 - loss: 0.7309
Epoch 10/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 0s 640us/step - accuracy: 0.8285 - loss: 0.6912


In [47]:
y_pred = model.predict(X_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 706us/step


In [49]:
y_pred = y_pred.argmax(axis = 1)

In [50]:
y_pred

array([7, 2, 1, ..., 9, 8, 6], dtype=int64)

In [52]:
accuracy = accuracy_score(test_labels, y_pred)
print("Accuracy:", accuracy)

confusion_matrix = confusion_matrix(test_labels, y_pred)
print("Confusion Matrix:")
print(confusion_matrix)

precision = precision_score(test_labels, y_pred, average='macro')
print("Precision:", precision)

recall = recall_score(test_labels, y_pred, average='macro')
print("Recall:", recall)

Accuracy: 0.8424
Confusion Matrix:
[[ 893    1   33    6    7   23    7    4    5    1]
 [   1 1090    3    5    3    3   12    4   14    0]
 [   9   17  849   52   17    7   31   19   21   10]
 [   8   10   69  811    3   30    9   32   35    3]
 [   1    2   10    1  812    6   11    3    7  129]
 [  40   10   18   50   26  634   34   24   45   11]
 [  16    7   23    3   16   18  872    0    3    0]
 [   6   27   28   11    3   10    0  895    6   42]
 [  13   20   19   31   31   37   21   34  757   11]
 [   6    7    8   13   87   11    2   55    9  811]]
Precision: 0.8407513042950543
Recall: 0.8396693581002257
